In [93]:
import geemap
import ee
import os
import datetime

# Spectral Linear Unmixing

In [94]:
Map = geemap.Map(center = [41.25827,28.928777],zoom=13)
Map.add_basemap('Esri Satellite')
Map


Map(center=[41.25827, 28.928777], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleBu…

In [95]:
aoi = ee.Geometry.Point([28.923747, 41.257073])
aoi.getInfo()

{'type': 'Point', 'coordinates': [28.923747, 41.257073]}

In [207]:
area = ee.Geometry.Polygon([[28.704534, 41.171665],
    [28.704534, 41.299722],
    [28.996758, 41.299722],
    [28.996758, 41.171665],
    [28.704534, 41.171665]])
area.getInfo()

{'type': 'Polygon',
 'coordinates': [[[28.704534, 41.171665],
   [28.996758, 41.171665],
   [28.996758, 41.299722],
   [28.704534, 41.299722],
   [28.704534, 41.171665]]]}

### Landsat Image

In [99]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")


bands = ['B2','B3','B4','B5','B6','B7']



image = landsat \
            .filterBounds(aoi) \
            .sort('CLOUD_COVER') \
            .filterDate('2020-05-01', '2020-05-30')\
            .first() \
            .select(bands)\
            .clip(area)
#             


landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.6,
    'min' : 0,
    'max' : 0.3
}

Map.addLayer(image, landsat_vis, "Landsat_RGB", True, 1)

### Sentinel Image

In [147]:
bands = ['B2','B3','B4','B5','B6','B7','B8','B11','B12']


def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

sentinel = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filterBounds(aoi) \
                .filterDate('2020-05-01', '2020-05-30') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                .map(maskS2clouds)\
                .select(bands)\
                .first()\
                .clip(area)

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

Map.addLayer(sentinel, visualization, 'Sentinel_RGB')

### Global Surface Water

In [238]:
gsw_dataset = ee.Image('JRC/GSW1_2/GlobalSurfaceWater')

gsw_dataset = gsw_dataset.clip(area)

visualization = {
  'bands': ['occurrence'],
  'min': 0.0,
  'max': 100.0,
  'palette': ['ffffff', 'ffbbbb', '0000ff']
};

Map.addLayer(gsw_dataset, visualization, 'Surface - Water');

### NDVI and NDWI 

In [222]:
visParams_ndvi = {'min': -0.2, 'max': 0.8, 'palette': 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000,529400,3E8601,207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'};

ndvi = sentinel.expression('(nir - red) / (nir + red)' ,{
    'nir' : sentinel.select('B8'),
    'red' : sentinel.select('B4')
})

ndwi = sentinel.expression('(nir - swir) / (nir + swir)' ,{
    'nir' : sentinel.select('B8'),
    'swir' : sentinel.select('B11')
})

Map.addLayer(ndwi, visParams_ndvi, 'Sentinel_NDWI_man')
Map.addLayer(ndvi, visParams_ndvi, 'Sentinel_NDVI_man')

## Image Stats

In [229]:
landsat_props = geemap.image_props(image)
sentinel_props = geemap.image_props(sentinel)
# landsat_props.getInfo()

In [230]:
Date = landsat_props.get('IMAGE_DATE').getInfo()
CC= landsat_props.get('CLOUD_COVER').getInfo()
Zone = landsat_props.get('UTM_ZONE').getInfo()
Scale = landsat_props.get('NOMINAL_SCALE').getInfo()
date_sentinel = sentinel.get('system:index').getInfo()
date_sentinel = datetime.datetime.strptime(date_sentinel.split('_')[0], "%Y%m%dT%H%M%S").date()

In [231]:
from pprint import pprint

Stats = {
    'Date' : Date,
    'Cloud Cover' : "{:.4} %".format(float(CC)*100),
    'UTM Zone ' : Zone, 
    'Resolution' : Scale
}

pprint(Stats)

{'Cloud Cover': '6.661e+03 %',
 'Date': '2020-05-14',
 'Resolution': 30,
 'UTM Zone ': 35}


### Create Area of Interest

In [232]:
area = Map.draw_last_feature

### Export Image

In [234]:
out_dir = '/home/cak/Desktop/lake_extraction/Data/exported'
filename = os.path.join(out_dir, f'{Date}_landsat_ist.tif')
filename_sentinel = os.path.join(out_dir, f'{date_sentinel}_sentinel_ist.tif')
filename_water = os.path.join(out_dir, 'surface_water_ist.tif')
filename_ndvi = os.path.join(out_dir, 'ndvi_ist.tif')
filename_ndwi = os.path.join(out_dir, 'ndwi_ist.tif')

In [184]:
image = image.clip(area).unmask()
sentinel = sentinel.clip(area).unmask()
ndvi = ndvi.clip(area).unmask()
ndwi = ndwi.clip(area).unmask()
geemap.ee_export_image(image, filename=filename, scale=30, file_per_band=False)
geemap.ee_export_image(sentinel, filename=filename_sentinel, scale=10, file_per_band=False)
geemap.ee_export_image(ndvi, filename=filename_ndvi, scale=10, file_per_band=False)
geemap.ee_export_image(ndwi, filename=filename_ndwi, scale=10, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /home/cak/Desktop/lake_extraction/Data/exported/2020-05-09_sentinel_ist.tif


In [251]:
filename_water = os.path.join(out_dir, 'surface_water_ist.tif')
gsw_dataset = gsw_dataset.clip(area).unmask()
geemap.ee_export_image(gsw_dataset, filename=filename_water, scale=20, file_per_band=False)


Generating URL ...
Please wait ...
Data downloaded to /home/cak/Desktop/lake_extraction/Data/exported/surface_water_ist.tif


### Add Ramsar to Map

In [13]:
ramsar_shp = '/home/cak/Desktop/lake_extraction/Data/sulakalanlar/ramsar_ist.shp'
ramsar = geemap.shp_to_ee(ramsar_shp)
Map.addLayer(ramsar, {}, 'Ramsar')

## Draw Endmembers

In [ ]:
# Map.draw_features

In [ ]:
# water_ee = ee.FeatureCollection(Map.draw_features)

In [ ]:
# forest_ee = ee.FeatureCollection(Map.draw_features)

In [ ]:
# soil_ee = ee.FeatureCollection(Map.draw_features)

In [ ]:
# geemap.ee_to_shp(water_ee, filename='./exported/water.shp')

In [ ]:
# geemap.ee_to_shp(forest_ee, filename='./exported/forest.shp')

In [ ]:
# geemap.ee_to_shp(soil_ee, filename='./exported/soil.shp')

### Read Endmembers

In [191]:
water_shp = '../Data/Endmembers/water_ist.shp'
water = geemap.shp_to_ee(water_shp)
Map.addLayer(water, {}, 'Water')

forest_shp = '../Data/Endmembers/forest_ist.shp'
forest = geemap.shp_to_ee(forest_shp)
Map.addLayer(forest, {}, 'forest')

soil_shp = '../Data/Endmembers/soil_ist.shp'
soil = geemap.shp_to_ee(soil_shp)
Map.addLayer(soil, {}, 'soil')

### Endmember Zonal Statistics

In [192]:
waterMean = sentinel.reduceRegion(
    **{
        'reducer': ee.Reducer.mean(),
        'geometry': water.geometry().getInfo(),
        'scale': 30,
        'maxPixels': 1e9
    }).values()

forestMean = sentinel.reduceRegion(
    **{
        'reducer': ee.Reducer.mean(),
        'geometry': forest.geometry().getInfo(),
        'scale': 30,
        'maxPixels': 1e9
    }).values()

soilMean = sentinel.reduceRegion(
    **{
        'reducer': ee.Reducer.mean(),
        'geometry': soil.geometry().getInfo(),
        'scale': 30,
        'maxPixels': 1e9
    }).values()

In [193]:
endmembers = ee.Array.cat([waterMean , forestMean , soilMean] , 1)

In [194]:
arrayImage = sentinel.toArray().toArray(1)

unmixed = ee.Image(endmembers).matrixSolve(arrayImage)

unmixedImage = unmixed.arrayProject([0]).arrayFlatten([['water','forest','soil']])

In [195]:
Map.addLayer(unmixedImage, {}, 'Unmixed')

In [14]:
waterBand = unmixedImage.select('water').gt(0.95)

Map.addLayer(waterBand, {} , 'waterBand')

In [15]:
classes = waterBand.reduceToVectors(
**{
  'reducer': ee.Reducer.countEvery(), 
  'geometry': waterBand.geometry().getInfo(), 
  'scale': 30,
  'maxPixels': 1e8
});


In [16]:
result = ee.FeatureCollection(classes);
Map.addLayer(result,{},'Water_Vector');

### Export Classification

In [196]:
filename = os.path.join(out_dir, f'{Date}_sentinel_unmixed_ist.tif')
geemap.ee_export_image(unmixedImage, filename=filename, scale=10, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /home/cak/Desktop/lake_extraction/Data/exported/2020-05-14_sentinel_unmixed_ist.tif


# Supervised classification


In [17]:
# clip image
# image_roi = image.clip(roi.geometry())
# Map.addLayer(image_roi,landsat_vis,'Cropped_Landsat')

In [235]:
waterf = ee.Feature(water.geometry(), {'class': 0, 'name': 'water'});
forestf = ee.Feature(forest.geometry(), {'class': 1, 'name': 'forest'});
soilf = ee.Feature(soil.geometry(), {'class': 2, 'name': 'soil'});

In [236]:
soilf

In [237]:
trainingFeatures = ee.FeatureCollection([waterf, forestf, soilf])

In [239]:
classifierTraining = sentinel.select(bands).sampleRegions(
      collection= trainingFeatures, 
      properties= ['class'], 
      scale= 30
    );

In [240]:
# // Randomly split the data into 60% for training, and 40% for testing
trainingTesting = classifierTraining.randomColumn('random',111009);

training = trainingTesting.filter(ee.Filter.lt('random', 0.6));

testing = trainingTesting.filter(ee.Filter.gte('random', 0.6));

### Non-linear regression functions

In [ ]:
cartclassifier = ee.Classifier.cart(randomSeed=111009).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    );

In [242]:
cartClasifficationImage = sentinel.select(bands).classify(cartclassifier);

Map.addLayer(cartClasifficationImage, {'min': 0, 'max': 2,
                                   'palette':['blue', 'green','yellow']},'CART classification');

In [243]:
rfClassification = ee.Classifier.smileRandomForest(numberOfTrees=1, seed=111009).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    )

In [244]:
# // Perform the RF regression on the landsat image
rfClassificationImage = sentinel.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification');

In [245]:
# ee.Classifier.libsvm(decisionProcedure, svmType, kernelType, shrinking, degree, gamma, coef0, cost, nu, terminationEpsilon, lossEpsilon, oneClass)

# // Create an SVM classifier with custom parameters.
svClassification = ee.Classifier.libsvm(kernelType='RBF',gamma=1,cost=100).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    )

In [246]:
# // Perform the RF regression on the landsat image
svClassificationImage = sentinel.select(bands).classify(svClassification);
    
# // Visualize the RF regression
Map.addLayer(svClassificationImage,{'min': 0, 'max': 2,
                                   'palette':['blue', 'green','yellow']}, 'SV CLassification');

### Export ML class

In [248]:
out_dir = '/home/cak/Desktop/lake_extraction/Data/exported'
filename_cart = os.path.join(out_dir, f'cart_ist.tif')
filename_rf = os.path.join(out_dir, 'rf_ist.tif')
filename_svm = os.path.join(out_dir, 'svm_ist.tif')

In [249]:
cartClasifficationImage = cartClasifficationImage.clip(area).unmask()
rfClassificationImage = rfClassificationImage.clip(area).unmask()
svClassificationImage = svClassificationImage.clip(area).unmask()
geemap.ee_export_image(cartClasifficationImage, filename=filename_cart, scale=10, file_per_band=False)
geemap.ee_export_image(rfClassificationImage, filename=filename_rf, scale=10, file_per_band=False)
geemap.ee_export_image(svClassificationImage, filename=filename_svm, scale=10, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /home/cak/Desktop/lake_extraction/Data/exported/cart_ist.tif
Generating URL ...
Please wait ...
Data downloaded to /home/cak/Desktop/lake_extraction/Data/exported/rf_ist.tif
Generating URL ...
Please wait ...
Data downloaded to /home/cak/Desktop/lake_extraction/Data/exported/svm_ist.tif
